In [317]:
# Import libraries and dependencies
import os
import requests
import numpy as np
import pandas as pd
import hvplot.pandas
import panel as pn
import plotly.express as px
import matplotlib.pyplot as plt
from pathlib import Path
from dotenv import load_dotenv
import alpaca_trade_api as tradeapi
import datetime 

%matplotlib inline

In [318]:
# Load .env enviroment variables
from dotenv import load_dotenv
load_dotenv()

# Set Alpaca API key and secret
alpaca_api_key = os.getenv("ALPACA_API_KEY")
alpaca_secret_key = os.getenv("ALPACA_SECRET_KEY")

# Create an object for aplpaca api 
api = tradeapi.REST(
    alpaca_api_key,
    alpaca_secret_key,
    api_version = "v2"
)


In [319]:
# Read all the sectors from the csv file 
sectors_csv = Path("Resources/spy_etf_sectors.csv")
sectors_df = pd.read_csv(sectors_csv)

sectors_df = sectors_df.drop(columns=["Zacks\nG Sector", "\nCompany", "% of\nPortfolio"])

sectors_df = sectors_df.rename(columns={
    "\nTicker": "tickers",
    "Russell\nSector": "sector"
})

In [320]:
# Get dxy data from barcharts csv

dxy_csv = Path("Resources/$dxy_historical_data_01-01_to_09-15-2020.csv")
dxy_df = pd.read_csv(dxy_csv)
dxy_df = dxy_df.rename(columns={"Last":"DXY", "Time":"date"})
dxy_df = dxy_df[["date", "DXY"]].dropna()

In [321]:
# Get bitcoin data from barcharts csv 

btc_csv = Path("Resources/btcusd_historical_data_01-01_to_09-15-2020.csv")
btc_df = pd.read_csv(btc_csv)
btc_df = btc_df.rename(columns={"Last":"BTC", "Time":"date"}).dropna()
btc_df = btc_df[["date", "BTC"]]

In [322]:
# Get gold data from barcharts csv 

xau_csv = Path("Resources/btcusd_historical_data_01-01_to_09-15-2020.csv")
xau_df = pd.read_csv(xau_csv)
xau_df = xau_df.rename(columns={"Last":"XAU", "Time":"date"}).dropna()
xau_df = xau_df[["date", "XAU"]]

In [323]:
# Combine bitcoin, gold, and dxy as alternative assets 

alternative_assets = xau_df.merge(btc_df, on='date')
alternative_assets = alternative_assets.merge(dxy_df, on='date')
alternative_assets['date'] = pd.to_datetime(alternative_assets['date'], utc=True).dt.date

In [324]:
# Set the ticker to stripes of 100
ticker_list = sectors_df["tickers"].to_list()
ticker_stripe_1 = ticker_list[0:99]
ticker_stripe_2 = ticker_list[99:198]
ticker_stripe_3 = ticker_list[198:297]
ticker_stripe_4 = ticker_list[297:396]
ticker_stripe_5 = ticker_list[396:495]
ticker_stripe_6 = ticker_list[495:]

# Set timeframe to '1D'
timeframe = "1D"

# Set start and end datetimes of from Jan 1 2020 to Sep 15 2020
start_date = start = pd.Timestamp("2020-01-01", tz="America/Chicago").isoformat()
end_date = pd.Timestamp("2020-09-15", tz="America/Chicago").isoformat()

# Chunk the data by stripes to make it compaitible with alpaca api ingestion 

spy_stripe_1 = api.get_barset(
    ticker_stripe_1,
    timeframe,
    start=start_date,
    end=end_date
).df

spy_stripe_2 = api.get_barset(
    ticker_stripe_2,
    timeframe,
    start=start_date,
    end=end_date
).df

spy_stripe_3 = api.get_barset(
    ticker_stripe_3,
    timeframe,
    start=start_date,
    end=end_date
).df

spy_stripe_4 = api.get_barset(
    ticker_stripe_4,
    timeframe,
    start=start_date,
    end=end_date
).df

spy_stripe_5 = api.get_barset(
    ticker_stripe_5,
    timeframe,
    start=start_date,
    end=end_date
).df

spy_stripe_6 = api.get_barset(
    ticker_stripe_6,
    timeframe,
    start=start_date,
    end=end_date
).df

# Combine all of the stripes together 
spy_df = pd.concat([spy_stripe_1, spy_stripe_2, spy_stripe_3, spy_stripe_4, spy_stripe_5, spy_stripe_6], axis=1)

In [325]:
# Write spy_df to csv to persist and to reformat date 

spy_df.to_csv('..Resources\spy_df_csv.csv', index = True)
spy_df_csv = Path('..Resources\spy_df_csv.csv')
spy_csv_df = pd.read_csv(spy_df_csv, header=None, index_col=None )
spy_csv_df[0] = pd.to_datetime(spy_csv_df[0], utc=True).dt.date

In [326]:
# Dropping the first to rows of multiindex and the other datapoints not needed 

spy_csv_df.columns = spy_csv_df.iloc[1]
spy_csv_df.drop(['open', 'high', 'low', 'volume'], axis=1, inplace=True)
spy_csv_df.columns = spy_csv_df.iloc[0]

In [327]:
# A lot of cleaning to get data outside of date to be a numeric 

spy_csv_df= spy_csv_df[2:]
spy_csv_df.columns.values[0]='date'

spy_csv_df = spy_csv_df.set_index('date')

spy_csv_df = spy_csv_df.apply(pd.to_numeric)

spy_csv_df = spy_csv_df.pct_change()

spy_csv_df = spy_csv_df.reset_index()

spy_csv_melt = spy_csv_df.melt(id_vars='date', var_name='tickers', value_name='close')


In [328]:
# Cleaning the alternative data to perform the pct_change function 

alternative_assets = alternative_assets.set_index('date')
alternative_assets = alternative_assets.pct_change()
alternative_assets = alternative_assets.reset_index()

# Melting to make consistent to spy_csv_df 

alternative_assets = alternative_assets.melt(id_vars='date', var_name='tickers', value_name='close')
alternative_assets['sector'] = 'ALTERNATIVE ASSETS'

In [329]:
final_df = spy_csv_melt.merge(sectors_df, on='tickers')

In [382]:
# A series of functions to return the plot that will be served on the panel 

def sector_mean():
    sector_mean = final_df.groupby(['date','sector'])['close'].mean().unstack()
    sector_mean_plot = sector_mean.hvplot(title="Average of Sector Returns")
    return sector_mean_plot

def sector_std():
    sector_std = final_df.groupby(['date','sector'])['close'].std().unstack()
    sector_std_plot = sector_std.hvplot(title="Standard Deviation of Sector Returns")
    return sector_std_plot

def sector_cumulative():
    sector_cum = (1 + final_df.groupby(['date','sector'])['close'].mean().unstack()).cumprod() - 1
    sector_cum_plot = sector_cum.hvplot(title="Cumulative Sector Returns")
    return sector_cum_plot

def alternative_cumulative():
    alternative_mean = (1 + alternative_assets.groupby(['date','sector'])['close'].mean().unstack()).cumprod() - 1
    alternative_mean_plot = alternative_mean.hvplot(title="Cumulative Alternative Asset Return")
    return alternative_mean_plot



In [383]:
# Creating a single tab to be served on the panel

welcome_column = pn.Column(
    "Birds Eye: This is a dashboard to see the top 500 symbols by sector. What you will get from this dashboard is a birds eye view of each sector, how they performed before COVID, during COVID and this month respectively",
    sector_cumulative,
    alternative_cumulative,
    
)

std_column = pn.Column(
    sector_std
)

mean_column = pn.Column(
    sector_mean,
    sizing_mode='scale_both'
)
panel = pn.Tabs(
    ("Birds Eye", welcome_column),
    ("Standard Deviaton", std_column),
    ("Average", mean_column)
)

In [384]:
# Birds eye, we have lift off

panel.servable()

Tabs
    [0] Column
        [0] Markdown(str)
        [1] Column
            [0] Column()
            [1] Row
                [0] HoloViews(NdOverlay, name='interactive140995')
        [2] Column
            [0] Column()
            [1] Row
                [0] HoloViews(Curve, name='interactive141268')
    [1] Column
        [0] Column
            [0] Column()
            [1] Row
                [0] HoloViews(NdOverlay, name='interactive141351')
    [2] Column(sizing_mode='scale_both')
        [0] Column
            [0] Column()
            [1] Row
                [0] HoloViews(NdOverlay, name='interactive141625')

In [385]:
sector_std()

:NdOverlay   [Variable]
   :Curve   [date]   (value)

In [386]:
sector_mean()

:NdOverlay   [Variable]
   :Curve   [date]   (value)